In [29]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import time
import json
import concurrent.futures 
from multiprocessing.dummy import Pool as pool
import re
import random
from lxml import etree
import pymongo
import threading
from tqdm import tqdm, trange
import jieba

## 获取所有分类标签

## 获取各个分类下的歌单id

In [21]:
# 读取json中数据
with open(filename) as f:
    my_ip_list = json.load(f)

NameError: name 'filename' is not defined

In [258]:
random.choice(my_ip_list)

'http://113.194.150.76:9999'

In [259]:
list1=[]
play_list_ids=[]
headers = {
    'Referer': 'http://music.163.com/',
    'Host': 'music.163.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.75 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'
    }

In [265]:
#通用的获取网站内容的框架
def getHTMLText(url,headers,ip): 
    try:
        proxies = {}
        if ip.split(":")[0] == 'http':
            proxies = {
                'http': ip
            }
        else:
            proxies = {
                'https': ip
            }
        r = rq.get(url,headers=headers,proxies=proxies)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "网络解析错误"

In [266]:
#解析每个广场页
def parse_main(html):
    soup=BeautifulSoup(html,'html.parser')
    c=soup.find_all('li')
    for unit in c:
        try:
            name_url=unit.find('a',{'class':"msk"})#m这里有URL，名字的信息
            # 正则获取歌单的id
            pattern = re.compile(r'\d+')
            id_str = pattern.findall(name_url['href'])
            play_list_ids.append(id_str[0])
        except:
            continue
    print("current counts of ids: " + str(len(play_list_ids)))

In [269]:
#获取首页该分类下面的歌单id
def get_playlist_ids(cat):
    depth=38
    start_url='https://music.163.com/discover/playlist/?order=hot&cat='+cat
    for i in range(depth):
        try:        
            url=start_url+'&limit=35'+'&offset='+str(35*(i))
            # 随机抽取ip
            ip = random.choice(my_ip_list)
            html=getHTMLText(url,headers,ip)
            parse_main(html)
            random_time = random.random()
            time.sleep(random_time * 4)
        except:
            print('失败')
            continue
    print(cat + " 标签爬取完毕")
    print("current counts of ids: " + str(len(play_list_ids)))
    print("========================")

In [270]:
# 根据标签爬取所有歌单id
for item in all_category:
    get_playlist_ids(item)

current counts of ids: 35
current counts of ids: 70
current counts of ids: 105
current counts of ids: 140
current counts of ids: 175
current counts of ids: 210
current counts of ids: 245
current counts of ids: 280
current counts of ids: 315
current counts of ids: 350
current counts of ids: 385
current counts of ids: 420
current counts of ids: 455
current counts of ids: 490
current counts of ids: 525
current counts of ids: 560
current counts of ids: 595
current counts of ids: 630
current counts of ids: 665
current counts of ids: 700
current counts of ids: 735
current counts of ids: 770
current counts of ids: 805
current counts of ids: 840
current counts of ids: 875
current counts of ids: 910
current counts of ids: 945
current counts of ids: 980
current counts of ids: 1015
current counts of ids: 1050
current counts of ids: 1085
current counts of ids: 1120
current counts of ids: 1155
current counts of ids: 1190
current counts of ids: 1225
current counts of ids: 1260
current counts of ids:

In [272]:
len(play_list_ids)

89761

In [271]:
# 将可用ip写入到json文件
filename = 'playlist_ids.json'
with open(filename, 'w') as fw:
    json.dump(play_list_ids, fw)

# 根据歌单id爬取歌单详细数据

In [38]:
# 加载所有歌单id
filename = 'playlist_ids.json'
with open(filename) as f:
    all_playlist_ids = json.load(f)

In [39]:
len(all_playlist_ids)

89761

In [40]:
class process_info:
    # 处理失败的歌单id
    failed_parse_ids = []

In [41]:
# 请求头
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

In [9]:
# 数据库连接与选择
myclient = pymongo.MongoClient("mongodb://192.168.206.100:27017/")
springTest_db = myclient["springTest"]
mycol = springTest_db["test_play_list"]

In [43]:
# 将数据放入mongodb中
def save_to_mongodb(info_dict):
    try:
        # 判断数据库是否已有该歌单
        result_count = mycol.count_documents({'_id':info_dict.get('_id')})
        if result_count == 0:
            mycol.insert_one(info_dict)
    except Exception as e:
        print("mongdb Error: " + e)

In [44]:
# 处理歌单详细信息的json
def parse_info_json(str_json, playlist_id):
    # 将json转换为dict
    data_dict = json.loads(str_json)
    # 获取所有的歌曲id
    track_ids = data_dict.get("playlist").get("trackIds")
    song_ids = []
    for track in track_ids:
        song_id = track.get('id')
        song_ids.append(song_id)
    # 歌单的标签列表
    tags_list = data_dict.get("playlist").get("tags")
    # 歌单名称
    name = data_dict.get("playlist").get("name")
    # 歌单订阅数量
    subscribedCount = data_dict.get("playlist").get("subscribedCount")
    # 歌单播放数量
    playCount = data_dict.get("playlist").get("playCount")
    # 将数据封装成dict
    playlist_info_dict = {
        "_id": playlist_id,
        "song_ids":song_ids,
        "tags_list": tags_list,
        "name": name,
        "subscribedCount": subscribedCount,
        "playCount": playCount,
    }
    return playlist_info_dict

In [60]:
#获取首页该分类下面的歌单id
def get_playlist_info(playlist_id):
    start_url='https://autumnfish.cn/playlist/detail?id=' + playlist_id
    try:        
        # 随机抽取ip
        ip = random.choice(current_ip_list)
        proxies = {}
        if ip.split(":")[0] == 'http':
            proxies = {
                'http': ip
            }
        else:
            proxies = {
                'https': ip
            }
        # 请求API获得json格式的字符串
        str_json = rq.get(url=start_url, proxies=proxies, headers=headers)
        # 解析数据
        playlist_info_dict = parse_info_json(str_json.text, playlist_id)
        # 将数据放入mongodb中
        save_to_mongodb(playlist_info_dict)
        # 随机间隔时间
#         random_time = random.random()
#         time.sleep(random_time * 2)
    except Exception as e:
        # 将处理失败的id添加入相应列表
        process_info.failed_parse_ids.append(playlist_id)
        print('Error: ' + str(e))

In [40]:
len(all_playlist_ids)

NameError: name 'all_playlist_ids' is not defined

In [65]:
with trange(len(all_playlist_ids)) as t:
    for i, item in enumerate(all_playlist_ids):
        get_playlist_info(item)
        t.set_description('Process %i' % i)  # 描述显示在左边
        t.update(1)

Process 19606:  22%|████████████▏                                           | 19607/89761 [2:35:28<13:00:40,  1.50it/s]

Error: Expecting value: line 1 column 1 (char 0)


Process 19607:  22%|████████████▏                                           | 19608/89761 [2:35:28<10:12:22,  1.91it/s]

Error: Expecting value: line 1 column 1 (char 0)


Process 19608:  22%|████████████▍                                            | 19609/89761 [2:35:29<8:13:30,  2.37it/s]

...

Process 83777:  93%|██████████████████████████████████████████████████████▏   | 83778/89761 [10:35:38<17:48,  5.60it/s]

Error: Expecting value: line 1 column 1 (char 0)


Process 83778:  93%|██████████████████████████████████████████████████████▏   | 83779/89761 [10:35:38<17:46,  5.61it/s]

Error: Expecting value: line 1 column 1 (char 0)


Process 89760: 100%|██████████████████████████████████████████████████████████| 89761/89761 [11:17:29<00:00,  2.21it/s]


In [62]:
start_time = time.time()
get_playlist_info(all_playlist_ids[4])
end_time = time.time()
print("运行时间："+str(end_time-start_time))

运行时间：0.4669215679168701


In [66]:
len(process_info.failed_parse_ids)

306

# 歌单数据处理

In [41]:
all_data = mycol.find()

In [43]:
all_data[0]

{'_id': '4991155974',
 'song_ids': [28427734,
  287677,
  409941447,
  402070840,
  409302060,
  27867475,
  407007445,
  233947,
  28341143,
  402073803,
  35847808,
  29723029,
  27867484,
  28406534,
  405377708,
  400876303,
  28427785,
  222621,
  29723039,
  28427777,
  26348068,
  26390020,
  400162549,
  31877581,
  28427771,
  26390019,
  26142919,
  25702226,
  28427763,
  403710394],
 'tags_list': ['流行', '综艺'],
 'name': '[最佳演出现场] 热门华语演出现场 线上听',
 'subscribedCount': 30512,
 'playCount': 4992275}

In [42]:
len(all_data)

TypeError: object of type 'Cursor' has no len()

In [32]:
mycol.count_documents({})

63835

In [11]:
all_count = all_data.collection.count()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


In [12]:
all_count

63835

In [13]:
all_data[0]

{'_id': '4991155974',
 'song_ids': [28427734,
  287677,
  409941447,
  402070840,
  409302060,
  27867475,
  407007445,
  233947,
  28341143,
  402073803,
  35847808,
  29723029,
  27867484,
  28406534,
  405377708,
  400876303,
  28427785,
  222621,
  29723039,
  28427777,
  26348068,
  26390020,
  400162549,
  31877581,
  28427771,
  26390019,
  26142919,
  25702226,
  28427763,
  403710394],
 'tags_list': ['流行', '综艺'],
 'name': '[最佳演出现场] 热门华语演出现场 线上听',
 'subscribedCount': 30512,
 'playCount': 4992275}

In [355]:
playlist_ids = []

In [356]:
for item in all_data:
    playlist_ids.append(item.get('_id'))

In [357]:
len(playlist_ids)

63835

In [358]:
playlist_ids[0]

'4991155974'

In [359]:
# 将歌单id写入到json文件
filename = './data/playlist_ids.json'
with open(filename, 'w', encoding='utf-8') as fw:
    json.dump(playlist_ids, fw)

In [72]:
all_song_ids = []
songs_count = 0

In [73]:
for item in all_data:
    for song_id in item.get('song_ids'):
        all_song_ids.append(song_id)

In [74]:
len(all_song_ids)

10101291

In [75]:
distinct_song_ids = set(all_song_ids)

In [76]:
len(distinct_song_ids)

1973840

In [77]:
import re

In [238]:
all_songs_dict = {}

In [239]:
with trange(all_count) as t:
    for item in all_data:
        song_ids = item.get("song_ids")
        tags_list = item.get("tags_list")
        name = item.get("name")
        for song_id in song_ids:
            song_dict = {}
            song_dict['song_id'] = song_id
            song_dict['tags'] = tags_list
            song_dict['play_lists'] = name
            # 该bgm的id已经存在
            if song_id in all_songs_dict.keys():
                # 将新的内容加入原数据中 
                orgin_tags = all_songs_dict[song_id].get('tags')
                orgin_play_lists = all_songs_dict[song_id].get('play_lists')
                # 合并标签列表
                new_tags = list(set(orgin_tags + tags_list))
                new_play_lists = name + ' ' + orgin_play_lists
                # 更新字典数据
                all_songs_dict[song_id]['tags'] = orgin_tags
                all_songs_dict[song_id]['play_lists'] = new_play_lists
            # 加入bgm列表
            else:
                all_songs_dict[song_id] = song_dict
        t.update(1)

100%|█████████████████████████████████████████████████████████████████████████▉| 63834/63835 [00:51<00:00, 1232.45it/s]


In [360]:
len(all_songs_dict.keys())

1973839

In [241]:
all_song_data = []

In [242]:
for item in all_songs_dict.values():
    all_song_data.append(item)

In [247]:
# 将可用ip写入到json文件
filename = 'all_song_data.json'
with open(filename, 'w', encoding='utf-8') as fw:
    json.dump(all_song_data, fw)

In [361]:
all_song_data[9872]

{'song_id': 1385408332,
 'tags': ['欧美', '流行', '综艺'],
 'play_lists': '消逝的神罚Music 积 跬 步 以 至 千 里 WanYYing 我将玫瑰藏于身后 时刻期盼着与你赴约 Kalinka 蹦迪 精神小伙儿精神依旧在 精选BMG/超嗨蹦迪 ♡、line.quan、 - dinosauR. ·! 【Russia Deep House】bass收录. 电音Ai_尔☛【DJ咕咚中文歌曲】非常的奈斯 大赦天下专用音乐 开始于朋友验证 结束于红色感叹号 封面选的好 绅士不会少 土嗨DJ 蹦迪选手已就位. DJ英文 纯音乐 电音车载音乐〖土嗨之王〗 土嗨/硬曲/网络BGM/300首 很潮很土很嗨很低調还盗版 【节奏控】DJ 蹦迪 撩人 入耳 激情 劲爆 重低音效果的DJ多多 Read All About It (Shoby House Rework) 『摇头｜蹦迪｜车载-仙曲』 我走在长街中，听戏子唱京城 你爹的资料好看吗 DJ FAKE 『一❤只蹦圣贤迪♪』 EA7-DJ精选歌单 A8L W12 酒吧摇头电音House - 全网最火的DJ 看见花瓣散漫了江面，你我相看缺不曾生厌 精神小伙，大赦天下 又好听却不知道歌名。 好听且得劲99 ♞[车载超重低音DJ] 一个暖男大叔的听歌列表 Dian Dian Dian yin 精选ξ电音易燃分享【持续更新】 欧美复古朋克节奏控 酷到沒朋友~樂 『硬曲 硬不硬听了再说（蹦迪专用❗）』 你绝对搜不到的土嗨神曲 EA7/康纳麦格雷戈 头一扭 曲一嗨 鬼火当成路虎开 法拉域Bass House匪气 嗨爆星空 dj-toxic顶级夜场嗨曲 热门劲爆DJ歌曲 深夜写作业艾薇巴蒂嗨起来(｢･ω･)｢ 【私藏】熟悉却忘记名字的电音 『精选』和别人炫耀过你...但还是丢了 精选～网红热歌 爸爸棒爸爸呗爸爸very 今夜 857 战术蹦迪 北执·ᴍᴜsɪᴄ ғᴍ ʀᴇᴍɪx -. 超嗨舞曲串烧！（苏卡比列） Dancers. 打游戏必备神曲 一生只蹦圣迪蹦 精神Dj全网最嗨，摇就完事了 精神小妹的社会歌 别把我听土嗨的事告诉别人～ 幹飯人.幹飯魂.乾飯都是人上人 长春奥斯卡DR.OSCAR 今天从《Tonight 》听起|私人雷达 你搜不到的神仙爆嗨歌单！！ 「DJ歌曲」最熟悉咚鼓版

In [363]:
# 修改key
for item in all_song_data:
    item['_id'] = item.pop('song_id')

In [364]:
all_song_data[9872]

{'tags': ['欧美', '流行', '综艺'],
 'play_lists': '消逝的神罚Music 积 跬 步 以 至 千 里 WanYYing 我将玫瑰藏于身后 时刻期盼着与你赴约 Kalinka 蹦迪 精神小伙儿精神依旧在 精选BMG/超嗨蹦迪 ♡、line.quan、 - dinosauR. ·! 【Russia Deep House】bass收录. 电音Ai_尔☛【DJ咕咚中文歌曲】非常的奈斯 大赦天下专用音乐 开始于朋友验证 结束于红色感叹号 封面选的好 绅士不会少 土嗨DJ 蹦迪选手已就位. DJ英文 纯音乐 电音车载音乐〖土嗨之王〗 土嗨/硬曲/网络BGM/300首 很潮很土很嗨很低調还盗版 【节奏控】DJ 蹦迪 撩人 入耳 激情 劲爆 重低音效果的DJ多多 Read All About It (Shoby House Rework) 『摇头｜蹦迪｜车载-仙曲』 我走在长街中，听戏子唱京城 你爹的资料好看吗 DJ FAKE 『一❤只蹦圣贤迪♪』 EA7-DJ精选歌单 A8L W12 酒吧摇头电音House - 全网最火的DJ 看见花瓣散漫了江面，你我相看缺不曾生厌 精神小伙，大赦天下 又好听却不知道歌名。 好听且得劲99 ♞[车载超重低音DJ] 一个暖男大叔的听歌列表 Dian Dian Dian yin 精选ξ电音易燃分享【持续更新】 欧美复古朋克节奏控 酷到沒朋友~樂 『硬曲 硬不硬听了再说（蹦迪专用❗）』 你绝对搜不到的土嗨神曲 EA7/康纳麦格雷戈 头一扭 曲一嗨 鬼火当成路虎开 法拉域Bass House匪气 嗨爆星空 dj-toxic顶级夜场嗨曲 热门劲爆DJ歌曲 深夜写作业艾薇巴蒂嗨起来(｢･ω･)｢ 【私藏】熟悉却忘记名字的电音 『精选』和别人炫耀过你...但还是丢了 精选～网红热歌 爸爸棒爸爸呗爸爸very 今夜 857 战术蹦迪 北执·ᴍᴜsɪᴄ ғᴍ ʀᴇᴍɪx -. 超嗨舞曲串烧！（苏卡比列） Dancers. 打游戏必备神曲 一生只蹦圣迪蹦 精神Dj全网最嗨，摇就完事了 精神小妹的社会歌 别把我听土嗨的事告诉别人～ 幹飯人.幹飯魂.乾飯都是人上人 长春奥斯卡DR.OSCAR 今天从《Tonight 》听起|私人雷达 你搜不到的神仙爆嗨歌单！！ 「DJ歌曲」最熟悉咚鼓版dj 尔兰超爱听的音乐 今天从《努力工作吧》听起

## 将歌曲信息写入MongoDB

In [4]:
# 数据库连接与选择
myclient = pymongo.MongoClient("mongodb://192.168.206.100:27017/")
springTest_db = myclient["springTest"]
song_data_collection = springTest_db["song_data"]

NameError: name 'pymongo' is not defined

In [366]:
song_data_collection.insert_one(all_song_data[9872])

In [396]:
result_data = song_data_collection.find({'_id':all_song_data[9872].get('_id')})

In [397]:
result_data.count()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


1

In [399]:
song_data_collection.update_one({'_id':all_song_data[9872].get('_id')}, {'$set': {'tags':['测试','都退']}})

In [385]:
# 加载所有歌曲标签
filename = './data/tag_words.json'
with open(filename, 'r', encoding='utf-8') as f:
    tag_words = json.load(f)

In [395]:
# 将歌单文本与标签列表进行对照
def parse_playlist_to_tags(str_playlists):
    tags = []
    # 对BGM标签的正则处理，替换
    pattern = re.compile(r'bgm|背景音乐', re.I)
    result = re.sub(pattern, 'BGM', str_playlists, 0)
    
    result = clean_zh_text(result)
    result = seg_sentence(result)
    data_list = list(set(result.split(' ')))
    # 返回清洗、分词、去重列表化的数据
    return data_list

In [394]:
parse_playlist_to_tags(all_song_data[9872].get('play_lists'))

['BGM', '搞笑', '经典', 'DJ', '纯音乐', '嗨', '古风', '游戏', '快乐', '蹦迪', '英文', '节奏', '伤感', '温柔', '电音', '踩点', '欧美']


In [405]:
def save_song_data_mongodb(song_data, name_collection, str_key='_id'):
    try:
        # 根据—_id的查询条件
        myquery_id = {'_id':song_data.get(str_key)}
        # 处理当前的歌单文本信息
        data_list = parse_playlist_to_tags(song_data.get('play_lists'))
        # 判断数据库是否已有该歌曲
        result_count = song_data_collection.count_documents(myquery_id)
        if result_count == 0:
            new_song_data = {}
            
            orgin_tags = song_data.get('tags')
            for item in data_list:
                if item in tag_words:
                    if item not in orgin_tags:
                        orgin_tags.append(item)
                        
            new_song_data['_id'] = song_data.get(str_key)
            new_song_data['tags'] = orgin_tags
            name_collection.insert_one(new_song_data)
        else:
            # 对已有的歌曲标签进行更新
            result_data = song_data_collection.find(myquery_id)
            orgin_data_dict = result_data[0]
            orgin_tags = orgin_data_dict.get('tags')
            
            for item in data_list:
                if item in tag_words:
                    if item not in orgin_tags:
                        orgin_tags.append(item)
            # mongodb修对应的改歌曲tags
            name_collection.update_one(myquery_id, {'$set': {'tags':orgin_tags}})
    except Exception as e:
        print('save_song_data_mongodb error:' + str(e))

In [407]:
with trange(len(all_song_data)) as t:
    for song_data in all_song_data:
        save_song_data_mongodb(song_data, song_data_collection)
        t.update(1)

100%|█████████████████████████████████████████████████████████████████████| 1973839/1973839 [2:38:20<00:00, 207.77it/s]


## BGM类型的歌曲处理

In [14]:
pattern = re.compile(r'bgm|背景|纯音乐', re.I)
bgm_playlist_list = []

In [15]:
for item in all_data:
    name = item.get("name")
    result = re.search(pattern=pattern, string=name)
    if result!=None:
        bgm_playlist_list.append(item)

In [16]:
len(bgm_playlist_list)

4472

In [17]:
bgm_playlist_list[0]

{'_id': '5074745281',
 'song_ids': [1313107065,
  28923579,
  29307041,
  484058025,
  1489146049,
  451126971,
  561307164,
  511412717,
  1313035063,
  26429479,
  1341509284,
  1456239624,
  1492891086,
  1338490248,
  21792987,
  1446035192,
  1355268293,
  31010790,
  558261233,
  1385738126,
  1486085481,
  509313150,
  29207817,
  442589034,
  506951906,
  1303289043,
  1460031009,
  529391087,
  34922517,
  531786301,
  522041157,
  1368324900,
  1474342935,
  29769258,
  26590043,
  4023379,
  1459793953,
  1479319438,
  1473560238,
  1398909242,
  1372082855,
  1439459693,
  1469868492,
  1380724680,
  1462368434,
  28864241,
  1306507078,
  1403078334,
  1360028990,
  1433945885,
  1356309192,
  1471064193,
  34040716,
  31719528,
  137473,
  439121233,
  5308028,
  310250,
  27639209,
  548633609,
  29737634,
  1470115202,
  863808572,
  32807378,
  1391281654,
  552145020,
  407889615,
  29561077,
  565833851,
  28283137,
  5180953,
  28785388,
  4978801,
  462179,
  33715

In [195]:
bgm_song_dict = {}

In [196]:
with trange(len(bgm_playlist_list)) as t:
    for item in bgm_playlist_list:
        song_ids = item.get("song_ids")
        tags_list = item.get("tags_list")
        name = item.get("name")
        for song_id in song_ids:
            song_dict = {}
            song_dict['song_id'] = song_id
            song_dict['tags'] = tags_list
            song_dict['play_lists'] = name
            # 该bgm的id已经存在
            if song_id in bgm_song_dict.keys():
                # 将新的内容加入原数据中 
                orgin_tags = bgm_song_dict[song_id].get('tags')
                orgin_play_lists = bgm_song_dict[song_id].get('play_lists')
                # 合并标签列表
                new_tags = list(set(orgin_tags + tags_list))
                new_play_lists = name + ' ' + orgin_play_lists
                # 更新字典数据
                bgm_song_dict[song_id]['tags'] = orgin_tags
                bgm_song_dict[song_id]['play_lists'] = new_play_lists
            # 加入bgm列表
            else:
                bgm_song_dict[song_id] = song_dict
        t.update(1)

100%|████████████████████████████████████████████████████████████████████████████| 4472/4472 [00:03<00:00, 1277.39it/s]


In [197]:
len(bgm_song_dict.keys())

237995

In [201]:
bgm_songs = []

In [227]:
bgm_song_dict[6]

KeyError: 6

In [204]:
for item in bgm_song_dict.values():
    bgm_songs.append(item)

In [257]:
len(bgm_songs)

237995

In [258]:
bgm_songs[990]

{'song_id': 32337414,
 'tags': ['浪漫', '流行', '综艺'],
 'play_lists': '心动的信号第三季BGM 心动的信号第三季BGM【已更新至第九期】 心动的信号第三季BGM'}

In [259]:
# 将可用ip写入到json文件
filename = 'bgm_data.json'
with open(filename, 'w', encoding='utf-8') as fw:
    json.dump(bgm_songs, fw)

In [22]:
filename = 'bgm_data.json'
with open(filename, encoding='utf-8') as f:
    bgm_json_data = json.load(f)

In [23]:
bgm_json_data[5]

{'song_id': 451126971,
 'tags': ['流行', '网络歌曲', '综艺'],
 'play_lists': "走秀专用歌单/时装视频剪辑背景乐 Rus&Me阿联酋 实战BGM 精选EA7 Bass 电音『Future bass』纯音乐 航拍延时摄影背景音乐合集② BGM异域风情 珍惜歌单 iMovie BGM List 丨BGM丨车载嗨曲 视频专用配乐背景音乐（纯音乐）无限更新 纯音乐（播音主持） 古风DJ纯音乐各种混搭 喜欢的可以看看 电耗子BGM（燃） 网易精选热门BGM HEINAN 旅拍油管《BGM》 节奏迷幻女声控BGM背景音乐英文和喜欢的 15S朋友圈短视频BGM 电影解说视频背景音乐精选 战疫纯音BGM 钢琴纯音乐＋其他纯音乐 精选.橙光|丧尸世界末日专属背景(更新10.17 那什么的剪辑BGM 禁忌女孩bgm们 时尚剪辑 | 纯音乐 出大事必备BGM 史诗级 短视频官方发布配音 『BGM』影视解说|VLOG|视频配音 经典背景音乐【精选】 【高级感秀场】T台走秀背景音乐精选 视频剪辑/纯音乐BGM 纯音乐游戏劲曲 王者|打野带节奏必备bgm|高燃（持续更新） 【全类型燃向BGM】GOD's love核爆音乐 动感·时尚·广告·宣传片背景乐高逼格 影视/动漫剪辑都可用的英文BGM 视频后期BGM 猫和老鼠背景音乐（主播专用） 纯音乐Bgm 每首都是精华 你能搜到的全网热歌♢BGM♢DJ♢热门D音 搞事情专用BGM 『BGM』你耳熟却又不知道名字的BGM 精选【bgm】 前后起伏纯音乐（TVC广告专用） 【VLOG踩点】超然视频BGM 『流行音乐BGM』大众喜爱歌单！ 皮皮搞笑很多人想要的BGM 精选| 超燃BGM！ 【黑帮】出场专用BGM 【写数学题专用】名侦探柯南破案BGM 「轻电音&纯音乐」(抖腿福利集) 燃，卡点剪辑BGM(大部分为高潮) Upper(剪辑bgm，开头，高潮) 『呕心沥血，史上最全』万首纯音乐大合集 【深夜读书】背景音乐 纯音乐/总会变好的 篮球男孩们你们的bgm来了记得带耳机(持更) 全网最火的音乐都在这《纯音乐》 向日葵[纯音乐] ✨SloMo宝藏BGM♡[开学少更了] 电音伴奏纯音乐(进来嗨) 悬疑-剧本杀BGM 战双帕弥什（战斗bgm） 百里守约BGM［疯狂失约］(º﹃º\xa0)阿巴~ 『铠

In [18]:
import CralwerProject.playlist_parse as playlist_parse

In [27]:
test = playlist_parse.parse_playlist_str(bgm_json_data[5].get('play_lists'))

In [28]:
print(test)

None


# 歌曲所在歌单信息解析

In [381]:
# 清洗数据，去除特殊字符
def clean_zh_text(text):
    # keep English, digital and Chinese
    comp = re.compile('[^A-Z^a-z^0-9^\u4e00-\u9fa5^\s^,]')
    return comp.sub('', text)

In [382]:
# 读取停用词表
def stopwordslist(filepath):
    stopwords=[line.strip() for line in open(filepath,'r',encoding='utf-8').readlines()]
    # strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
    #readlines()读取所有行并返回列表
    return stopwords

In [411]:
#对句子进行分词
def seg_sentence(sentence):
    # 去除特殊字符
    clean_zh_text(sentence)
    sentence_seged=jieba.cut(sentence.strip())
    stopwords=stopwordslist(r"./data/stopwords_cn.txt")#这里加载停用词的路径
    outstr=""
    for word in sentence_seged:
        if word not in stopwords:
            if word!='\t':
                outstr+=word
                outstr+=" "
    return outstr

In [262]:
result = clean_zh_text(bgm_json_data[5].get('play_lists')) 

In [379]:
result = result.split(' ')

AttributeError: 'list' object has no attribute 'split'

In [412]:
bgm_json_data[5].get('play_lists')

"走秀专用歌单/时装视频剪辑背景乐 Rus&Me阿联酋 实战BGM 精选EA7 Bass 电音『Future bass』纯音乐 航拍延时摄影背景音乐合集② BGM异域风情 珍惜歌单 iMovie BGM List 丨BGM丨车载嗨曲 视频专用配乐背景音乐（纯音乐）无限更新 纯音乐（播音主持） 古风DJ纯音乐各种混搭 喜欢的可以看看 电耗子BGM（燃） 网易精选热门BGM HEINAN 旅拍油管《BGM》 节奏迷幻女声控BGM背景音乐英文和喜欢的 15S朋友圈短视频BGM 电影解说视频背景音乐精选 战疫纯音BGM 钢琴纯音乐＋其他纯音乐 精选.橙光|丧尸世界末日专属背景(更新10.17 那什么的剪辑BGM 禁忌女孩bgm们 时尚剪辑 | 纯音乐 出大事必备BGM 史诗级 短视频官方发布配音 『BGM』影视解说|VLOG|视频配音 经典背景音乐【精选】 【高级感秀场】T台走秀背景音乐精选 视频剪辑/纯音乐BGM 纯音乐游戏劲曲 王者|打野带节奏必备bgm|高燃（持续更新） 【全类型燃向BGM】GOD's love核爆音乐 动感·时尚·广告·宣传片背景乐高逼格 影视/动漫剪辑都可用的英文BGM 视频后期BGM 猫和老鼠背景音乐（主播专用） 纯音乐Bgm 每首都是精华 你能搜到的全网热歌♢BGM♢DJ♢热门D音 搞事情专用BGM 『BGM』你耳熟却又不知道名字的BGM 精选【bgm】 前后起伏纯音乐（TVC广告专用） 【VLOG踩点】超然视频BGM 『流行音乐BGM』大众喜爱歌单！ 皮皮搞笑很多人想要的BGM 精选| 超燃BGM！ 【黑帮】出场专用BGM 【写数学题专用】名侦探柯南破案BGM 「轻电音&纯音乐」(抖腿福利集) 燃，卡点剪辑BGM(大部分为高潮) Upper(剪辑bgm，开头，高潮) 『呕心沥血，史上最全』万首纯音乐大合集 【深夜读书】背景音乐 纯音乐/总会变好的 篮球男孩们你们的bgm来了记得带耳机(持更) 全网最火的音乐都在这《纯音乐》 向日葵[纯音乐] ✨SloMo宝藏BGM♡[开学少更了] 电音伴奏纯音乐(进来嗨) 悬疑-剧本杀BGM 战双帕弥什（战斗bgm） 百里守约BGM［疯狂失约］(º﹃º\xa0)阿巴~ 『铠战歌（逆风战歌）超燃节奏BGM』 粤知一二BGM 情感电台·读文BGM·治愈系 【BGM】动感时尚大气纯音乐 【日常实用】Pr视频剪辑适用

In [291]:
result = bgm_json_data[5].get('play_lists').split(' ')


In [292]:
test

'走秀 专用 歌单 时装 视频剪辑 背景 乐 RusMe 阿联酋 实战 BGM 精选 EA7 Bass 电音 Future bass 纯音乐 航拍 延时 摄影 背景音乐 合集 BGM 异域 风情 珍惜 歌单 iMovie BGM List 丨 BGM 丨 车载 嗨 曲 视频 专用 配乐 背景音乐 纯 音乐无限 更新 纯音乐 播音 主持 古风 DJ 纯音乐 各种 混 搭 喜欢 看看 电 耗子 BGM 燃 网易 精选 热门 BGM HEINAN 旅拍 油管 BGM 节奏 迷幻 女 声控 BGM 背景音乐 英文 喜欢 15S 朋友圈 短 视频 BGM 电影 解说 视频 背景音乐 精选 战疫 纯音 BGM 钢琴 纯音乐 纯音乐 精选 橙光 丧尸 世界末日 专属 背景 更新 1017 剪辑 BGM 禁忌 女孩 bgm 时尚 剪辑 纯音乐 出 大事 必备 BGM 史诗 级 短 视频 官方 发布 配音 BGM 影视 解说 VLOG 视频 配音 经典 背景音乐 精选 高级 感秀场 T台 走秀 背景音乐 精选 视频剪辑 纯音乐 BGM 纯音乐 游戏 劲曲 王者 打野 带 节奏 必备 bgm 高燃 持续 更新 全 类型 燃向 BGMGODs love 核爆 音乐 动感 时尚 广告 宣传片 背景 乐高 逼格 影视 动漫 剪辑 都 可用 英文 BGM 视频 后期 BGM 猫和老鼠 背景音乐 主播 专用 纯音乐 Bgm 首都 精华 能 搜到 全网 热歌 BGMDJ 热门 D 音 搞 事情 专用 BGM BGM 耳熟 知道 名字 BGM 精选 bgm 前后 起伏 纯音乐 TVC 广告 专用 VLOG 踩点 超然 视频 BGM 流行音乐 BGM 大众 喜爱 歌单 皮皮 搞笑 很多 想要 BGM 精选 超燃 BGM 黑帮 出场 专用 BGM 写 数学题 专用 名 侦探 柯南 破案 BGM 轻电音 纯音乐 抖 腿 福利 集 燃卡点 剪辑 BGM 大部分 高潮 Upper 剪辑 bgm 开头 高潮 呕心沥血 史上 最全 万首 纯音乐 大 合集 深夜 读书 背景音乐 纯音乐 总会 变好 篮球 男孩 bgm 记得 带 耳机 持 更 全网 最火 音乐 都 纯音乐 向日葵 纯音乐 SloMo 宝藏 BGM 开学 少 更 电音 伴奏 纯音乐 进来 嗨 悬疑剧 本杀 BGM 战双 帕弥什 战斗 bgm 百里 守约

In [420]:
result = bgm_json_data[5].get('play_lists').split(' ')
output = playlist_parse.parse_playlist_str(result)

AttributeError: module 'CralwerProject.playlist_parse' has no attribute 'parse_playlist_str'

In [286]:
seg=jieba.cut("李小福是创新办主任也是云计算方面的专家, 我不是专家BOSSas哈哈")#默认精确模式 也可以设置cut_all=False
print("/".join(seg))

李小福/是/创新/办/主任/也/是/云/计算/方面/的/专家/,/ /我/不是/专家/BOSSas/哈哈


In [265]:
all_playlist_data = ''

In [274]:
for item in bgm_json_data:
    all_playlist_data += item.get('play_lists')

In [275]:
# 去除特殊字符
all_playlist_data = clean_zh_text(all_playlist_data)

In [295]:
output_data = ''
all_playlist_data = all_playlist_data.split(' ')

In [297]:
with trange(len(all_playlist_data)) as t:
    for sentence in all_playlist_data:
        output_data += seg_sentence(sentence)
        t.update(1)

100%|████████████████████████████████████████████████████████████████████████| 852185/852185 [09:33<00:00, 1486.82it/s]


In [298]:
# 将处理好的歌单文本写入到文件
filename = './data/all_playlist_data.txt'
with open(filename, 'w', encoding='utf-8') as fw:
    json.dump(output_data, fw)

In [300]:
from collections import Counter

In [301]:
word_freq = dict(Counter(output_data.split(' ')))

In [306]:
# 根据value将字典降序排序
sorted_word_freq = sorted(word_freq.items(), key=lambda d: d[1], reverse=True)

In [312]:
type(sorted_word_freq[1])

tuple

In [315]:
df.to_excel('./data/sorted_word_freq.xlsx', index=False)

In [316]:
word_df = pd.read_excel('./data/words_dict.xlsx')

In [319]:
tag_words = word_df['word']

In [326]:
tag_words = tag_words.values.tolist()

In [328]:
tag_words.remove('视频剪辑')

In [330]:
tag_words.remove('燃向')

In [332]:
filename = './data/tag_words.json'
with open(filename, 'w', encoding='utf-8') as fw:
    json.dump(tag_words, fw,ensure_ascii=False,indent = 4)